In [1]:
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os
%matplotlib inline

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

####### 修改输入文件
filename = 'acc_100_36.h5'
file = h5py.File(os.path.join('data/',filename),'r')
imageData   = file['imageData'][:]
imageLabel  = file['imageLabel'][:]  
file.close()

# 随机打乱数据和标签
N = imageData.shape[0]
index = np.random.permutation(N)
data  = imageData[index,:,:]
label = imageLabel[index]

# 对数据升维,标签one-hot
# data  = np.expand_dims(data, axis=1)
label = convert_to_one_hot(label,49).T
label = np.argmax(label, axis=1)  

# 划分数据集
N = data.shape[0]
num_train = round(N*0.8)
X_train = data[0:num_train,:,:]
Y_train = label[0:num_train]
X_test  = data[num_train:N,:,:]
Y_test  = label[num_train:N]

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

/home/data_mining/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


X_train shape: (19162, 100, 36)
Y_train shape: (19162,)
X_test shape: (4791, 100, 36)
Y_test shape: (4791,)


In [2]:
class dataset_prediction(Dataset):
    def __init__(self, data_features, data_target):
        self.len = len(data_features)
        self.features = torch.from_numpy(data_features)
        self.target = torch.from_numpy(data_target)
        
    def __getitem__(self, index):
        return self.features[index], self.target[index]

    def __len__(self):
        return self.len

train_set = dataset_prediction(data_features=X_train, data_target=Y_train)
test_set = dataset_prediction(data_features=X_test, data_target=Y_test)

train_loader = DataLoader(dataset=train_set,
                            batch_size=64,
                            shuffle=True,
                            drop_last=True)

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = torch.tensor(x)
        x = x.to(torch.float32)
        x = x.cuda()
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [4]:
input_size = 36
hidden_size = 64
num_layers = 4
num_classes = 49

# lstm 实例化
lstm = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)
print(lstm)

if torch.cuda.is_available():
    lstm = lstm.cuda()

epoches = 100
learning_rate = 0.001

# 记录训练参数
train_losses = []
test_accuracies = []

# 定义优化器和损失函数
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_function = loss_function.cuda()

# 开始训练
for epoch in range(epoches):
    print("进行第{}个epoch".format(epoch))
    for step, (batch_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        
        output = lstm(batch_x)

        loss = loss_function(output, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 为了实时显示准确率
        if step % 64 == 0:
            test_output = lstm(X_test).cpu()
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float(np.sum(pred_y == Y_test)) / float(Y_test.shape[0])
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.cpu().data.numpy(), '| test accuracy: %.2f' % accuracy)
    # 保存训练损失和测试准确率
    train_losses.append(loss.cpu().data.numpy())
    test_accuracies.append(accuracy)

test_output = lstm(X_test[:10]).cpu()
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y)
print(X_test[:10])


LSTM(
  (lstm): LSTM(36, 64, num_layers=4, batch_first=True)
  (fc): Linear(in_features=64, out_features=49, bias=True)
)
进行第0个epoch
Epoch:  0 | train loss: 3.8968 | test accuracy: 0.02


/home/data_mining/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Epoch:  0 | train loss: 3.8190 | test accuracy: 0.04
Epoch:  0 | train loss: 3.5963 | test accuracy: 0.04
Epoch:  0 | train loss: 3.4829 | test accuracy: 0.06
Epoch:  0 | train loss: 3.3868 | test accuracy: 0.06
进行第1个epoch
Epoch:  1 | train loss: 3.2170 | test accuracy: 0.06
Epoch:  1 | train loss: 2.9530 | test accuracy: 0.11
Epoch:  1 | train loss: 3.2117 | test accuracy: 0.10
Epoch:  1 | train loss: 2.7793 | test accuracy: 0.11
Epoch:  1 | train loss: 2.7758 | test accuracy: 0.12
进行第2个epoch
Epoch:  2 | train loss: 2.9042 | test accuracy: 0.14
Epoch:  2 | train loss: 2.9939 | test accuracy: 0.15
Epoch:  2 | train loss: 2.6836 | test accuracy: 0.16
Epoch:  2 | train loss: 2.7263 | test accuracy: 0.17
Epoch:  2 | train loss: 2.6165 | test accuracy: 0.16
进行第3个epoch
Epoch:  3 | train loss: 2.5968 | test accuracy: 0.19
Epoch:  3 | train loss: 2.9146 | test accuracy: 0.11
Epoch:  3 | train loss: 2.7666 | test accuracy: 0.14
Epoch:  3 | train loss: 2.9804 | test accuracy: 0.11
Epoch:  3 | t

In [ ]:
print(test_output)

In [7]:
import numpy as np
print("max_test_accuracy:{:.2f}%".format(np.median(np.array(test_accuracies))*100))

max_test_accuracy:79.15%


In [ ]:
# 绘制结果
#保存训练损失和测试准确率图表
import os
current_dir = os.getcwd()
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(train_losses,label='Train Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Train Loss over Iterations')
plt.subplot(1,2,2)
plt.plot(test_accuracies,label='Test Accuracy')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Test Accuracy over Iterations')
plt.savefig(os.path.join(current_dir,'runs/experiment2/',filename+'.png'))
# plt.savefig(os.path.join(current_dir,'runs/experiment1/','emg_100_12'+'.png'))
plt.show()

In [ ]:
# 假设 model 是你的 CNN 模型
torch.save(lstm.state_dict(), 'model/weight/lstm/'+filename+'.pth')
